<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

## Step 1.1: Configure the environment

In [1]:
from datetime import datetime
from shapely.geometry import Polygon
from shapely import box
from pyproj import CRS 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
from ipyleaflet import basemaps
from numpy.lib.stride_tricks import sliding_window_view
import pystac_client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import planetary_computer
from odc.stac import load
from pystac.client import Client
from skimage.feature import graycomatrix, graycoprops
from utils import load_data, scale, calculate_band_indices, apply_mask, mask_land, mask_deeps, mask_elevation, all_masks, glcm_features, do_prediction

In [2]:
# Reload scripts and imports
%load_ext autoreload
%autoreload 2

In [3]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Enter your site name
site = "bootless"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

nm-bootless-16072025


## Step 1.2: Configure STAC access and search parameters

In [4]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [5]:
# ## Use training data bounds

training = gpd.read_file("training-data/nm-bootless-bay-13072025_postcard.geojson")
training = training.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = training.total_bounds

bbox = [min_lon, min_lat, max_lon, max_lat]

# TODO: configure colours...
# training.explore(column="observed")

In [6]:
datetime = "2024"

items = client.search(
    collections=["dep_s2_geomad"],
    datetime=datetime,
    bbox=bbox
).item_collection()

print(f"Found {len(items)} items in for {datetime}")

Found 1 items in for 2024


In [7]:
measurements = ["nir", "red", "blue", "green", "emad", "smad", "bcmad", "green", "nir08", "nir09", "swir16", "swir22", "coastal", "rededge1", "rededge2", "rededge3"]
data = load_data(
    items,
    measurements,
    bbox,
)
    
# Now you can use the 'data' variable
print(data)

<xarray.Dataset> Size: 5MB
Dimensions:      (y: 490, x: 262, time: 1)
Coordinates:
  * y            (y) float64 4kB -1.055e+06 -1.055e+06 ... -1.06e+06 -1.06e+06
  * x            (x) float64 2kB -3.036e+05 -3.036e+05 ... -3.01e+05 -3.01e+05
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/16)
    nir          (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    red          (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    blue         (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    green        (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    emad         (time, y, x) float32 514kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    smad         (time, y, x) float32 514kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 257kB dask.arr

In [8]:
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
configure_s3_access(cloud_defaults=True, requester_pays=True)

In [9]:
scaled = scale(data)
scaled = scaled.compute().squeeze()

In [10]:
# Explore the site we are working on
scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [11]:
scaled

<xarray.Dataset> Size: 15MB
Dimensions:      (y: 490, x: 262)
Coordinates:
  * y            (y) float64 4kB -1.055e+06 -1.055e+06 ... -1.06e+06 -1.06e+06
  * x            (x) float64 2kB -3.036e+05 -3.036e+05 ... -3.01e+05 -3.01e+05
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/16)
    nir          (y, x) float64 1MB 0.2229 0.2008 0.2342 ... 0.0526 0.0526
    red          (y, x) float64 1MB 0.0638 0.0532 0.046 ... 0.0569 0.056 0.0565
    blue         (y, x) float64 1MB 0.0496 0.0425 0.0389 ... 0.0751 0.0751
    green        (y, x) float64 1MB 0.071 0.0589 0.055 ... 0.0703 0.0701 0.0704
    emad         (y, x) float32 514kB 0.1151 0.1159 0.1175 ... 0.1177 0.1171
    smad         (y, x) float32 514kB 1.245e-06 1.374e-06 ... 1.31e-06 1.308e-06
    ...           ...
    swir16       (y, x) float64 1MB 0.2136 0.2119 0.2408 ... 0.0537 0.0537
    swir22       (y, x) float64 1MB 0.1374 0.1355 0.1691 ... 0.0501 0.0501
    coastal      (y, x) float64 1MB 0.0532 0.0521 0.0523 ... 0.0699 0.069 0.0689
    rededge1     (y, x) float64 1MB 0.1035 0.1027 0.1275 ... 0.0595 0.0594
    rededge2     (y, x) float64 1MB 0.1915 0.1912 0.2172 ... 0.0564 0.0564
    rededge3     (y, x) float64 1MB 0.2226 0.2232 0.2496 ... 0.059 0.058 0.058

In [12]:
scaled = calculate_band_indices(scaled)
Dataset = scaled

In [13]:
land_masked_ds = mask_land(scaled, return_mask = True)
# land_masked_ds.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [14]:
deeps_masked_ds, deeps_mask = mask_deeps(scaled, return_mask = True)
deeps_mask.odc.explore(name=site)
# scaled_masked.red.odc.explore()

In [15]:
elevation_masked_ds, elevation_mask = mask_elevation(scaled, return_mask = True)
elevation_mask.odc.explore(name=site)

In [16]:
# e84_catalog = "https://earth-search.aws.element84.com/v1/"
# e84_client = Client.open(e84_catalog)
# collection = "cop-dem-glo-30"

# items = e84_client.search(
#     collections=[collection],
#     bbox=bbox
# ).item_collection()

# items

# # Using geobox means it will load the elevation data the same shape as the other data
# elevation = load(items, measurements=["data"], geobox=data.odc.geobox).squeeze()

# # True where data is above 10m elevation
# elevation_mask = elevation.data < 10

# # elevation_mask.odc.explore()

In [17]:
masked_scaled, mask = all_masks(scaled, return_mask = True)
mask.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [18]:
masked_scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

/srv/conda/envs/notebook/lib/python3.10/site-packages/odc/geo/_rgba.py:55: RuntimeWarning: invalid value encountered in cast
  return x.astype("uint8")


In [19]:
# Now apply the mask, and view the masked region

masked_scaled = masked_scaled.where(all_masks)
masked_scaled.odc.explore(
    vmin=0, vmax=0.3, bands=["red", "green", "blue"], name=f"{site}-masked", tiles=basemaps.Esri.WorldImagery
)

/srv/conda/envs/notebook/lib/python3.10/site-packages/odc/geo/_rgba.py:55: RuntimeWarning: invalid value encountered in cast
  return x.astype("uint8")


### GLCM texture analysis

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [20]:
WINDOW_SIZE = 9
LEVELS = 32

# Input
max = masked_scaled.blue.max().values
min = masked_scaled.blue.min().values
# Scale to 0-LEVELS for GLCM
img = ((masked_scaled.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)

# Extract overlapping windows
patches = sliding_window_view(img, (WINDOW_SIZE, WINDOW_SIZE))
# Shape: (rows, cols, win_y, win_x)


/tmp/ipykernel_700/4208043717.py:8: RuntimeWarning: invalid value encountered in cast
  img = ((masked_scaled.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)


In [21]:
import numpy as np # Ensure numpy is imported if not already

# Assuming 'patches' is a 4D NumPy array with dimensions (y_coords, x_coords, window_y_size, window_x_size)
# To get the first patch (at y=0, x=0), you would index it like this:
sample_patch_data = patches[0, 0, :, :]

# Verify the shape of the extracted sample patch data
print(f"Shape of sample_patch_data: {sample_patch_data.shape}")

# Call glcm_features directly on this 2D sample data
sample_result = glcm_features(sample_patch_data)

# Print the shape of the result to get the number of features
print(f"Shape of glcm_features output for a single patch: {sample_result.shape}")

Shape of sample_patch_data: (9, 9)
Shape of glcm_features output for a single patch: (7,)


In [22]:
# Use apply_ufunc to vectorize over (row, col) dimensions
result = xr.apply_ufunc(
    glcm_features,
    xr.DataArray(patches, dims=["y", "x", "win_y", "win_x"]),
    input_core_dims=[["win_y", "win_x"]],
    output_core_dims=[["feature"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float32]
)

# Add coordinates & names
pad = WINDOW_SIZE - 1
result = result.assign_coords({
    "y": masked_scaled.y[: -pad],
    "x": masked_scaled.x[: -pad],
    "feature": ["contrast", "homogeneity", "energy", "ASM", "correlation", "mean", "entropy"]
})

result_bands = result.to_dataset(dim="feature")

# Combine with original
masked_plus = masked_scaled.copy()
masked_plus = masked_plus.assign(result_bands)

masked_plus

<xarray.Dataset> Size: 31MB
Dimensions:      (y: 490, x: 262)
Coordinates:
  * y            (y) float64 4kB -1.055e+06 -1.055e+06 ... -1.06e+06 -1.06e+06
  * x            (x) float64 2kB -3.036e+05 -3.036e+05 ... -3.01e+05 -3.01e+05
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/35)
    nir          (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    red          (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    blue         (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    green        (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    emad         (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    smad         (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    ...           ...
    homogeneity  (y, x) float32 514kB 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan
    energy       (y, x) float32 514kB 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan
    ASM          (y, x) float32 514kB 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan
    correlation  (y, x) float32 514kB 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan
    mean         (y, x) float32 514kB 0.0 0.0 0.0 0.0 0.0 ... nan nan nan nan
    entropy      (y, x) float32 514kB -1.443e-10 -1.443e-10 ... nan nan

In [23]:
# Re-apply the mask
masked_plus = masked_plus.where(all_masks)

### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [24]:
# Reproject training data to the GeoMAD CRS and convert to xarray
training_reprojected = training.to_crs(masked_plus.odc.crs)
training_da = training_reprojected.assign(
    x=training_reprojected.geometry.x, y=training_reprojected.geometry.y
).to_xarray()

# Extract training values from the masked dataset
training_values = (
    masked_plus.sel(training_da[["x", "y"]], method="nearest")
    .squeeze()
    .compute()
    .to_pandas()
)
training_values

,nir,red,blue,green,emad,smad,bcmad,count,nir08,nir09,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
index,,,,,,,,,,,,,,,,,,,,,
0,0.0602,0.0742,0.0926,0.1005,0.152102,0.000002,0.000034,0.0031,0.0637,0.0988,...,0.888889,0.575007,0.330633,0.552448,3.541667,1.757751,-1057915.0,-302095.0,3832,2024-01-01
1,0.0598,0.0750,0.0989,0.1071,0.161220,0.000003,0.000034,0.0031,0.0648,0.0689,...,0.868056,0.661656,0.437789,0.282831,3.756944,1.547091,-1057915.0,-302075.0,3832,2024-01-01
2,0.0597,0.0750,0.0949,0.1040,0.161838,0.000003,0.000034,0.0031,0.0667,0.0682,...,0.951389,0.839043,0.703993,0.533117,3.881944,0.910556,-1057945.0,-302075.0,3832,2024-01-01
3,0.0597,0.0752,0.0952,0.1040,0.158307,0.000002,0.000034,0.0031,0.0662,0.0687,...,0.895833,0.636696,0.405382,0.515152,3.687500,1.604869,-1057925.0,-302085.0,3832,2024-01-01
4,0.0580,0.0723,0.0995,0.1074,0.159002,0.000003,0.000034,0.0031,0.0638,0.0677,...,0.875000,0.694722,0.482639,0.242105,3.791667,1.438722,-1057915.0,-302065.0,3832,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.811111,0.626541,0.392554,0.635212,0.819444,2.166396,-1059575.0,-303115.0,3832,2024-01-01
816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.805556,0.536028,0.287326,0.673772,1.208333,2.328758,-1059535.0,-303115.0,3832,2024-01-01
817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.811111,0.505181,0.255208,0.681997,1.569444,2.311211,-1059485.0,-303115.0,3832,2024-01-01


In [25]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["cc_id"], training_values], axis=1)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

,cc_id,nir,red,blue,green,emad,smad,bcmad,count,nir08,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
0,4,0.0602,0.0742,0.0926,0.1005,0.152102,0.000002,0.000034,0.0031,0.0637,...,0.888889,0.575007,0.330633,0.552448,3.541667,1.757751,-1057915.0,-302095.0,3832,2024-01-01
1,4,0.0598,0.0750,0.0989,0.1071,0.161220,0.000003,0.000034,0.0031,0.0648,...,0.868056,0.661656,0.437789,0.282831,3.756944,1.547091,-1057915.0,-302075.0,3832,2024-01-01
2,4,0.0597,0.0750,0.0949,0.1040,0.161838,0.000003,0.000034,0.0031,0.0667,...,0.951389,0.839043,0.703993,0.533117,3.881944,0.910556,-1057945.0,-302075.0,3832,2024-01-01
3,4,0.0597,0.0752,0.0952,0.1040,0.158307,0.000002,0.000034,0.0031,0.0662,...,0.895833,0.636696,0.405382,0.515152,3.687500,1.604869,-1057925.0,-302085.0,3832,2024-01-01
4,4,0.0580,0.0723,0.0995,0.1074,0.159002,0.000003,0.000034,0.0031,0.0638,...,0.875000,0.694722,0.482639,0.242105,3.791667,1.438722,-1057915.0,-302065.0,3832,2024-01-01


In [26]:
print(training_array.shape[1], 'total columns')
print('columns included', training_array.columns)

40 total columns
columns included Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'count', 'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi',
       'ndwi', 'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x',
       'spatial_ref', 'time'],
      dtype='object')


In [27]:
standard_schema = ['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'count', 'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi',
       'ndwi', 'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x', 'time',
       'spatial_ref']

In [28]:
training_array=training_array[standard_schema]

In [29]:
training_array=training_array.drop(columns=["spatial_ref", "time", "count"])

In [30]:
# Write the training data to a CSV file
training_array.to_csv(f"training-data/{version}-training.csv", index=False)